# Iterating Tables/Sheets

Here we'll talk about working with iterable data sources - such as extracting and joining data taken from multiple tables from a single spreadheet.


## Source Data

The data source we're using for these examples is shown below:

| <span style="color:green">Note - this particular table has some very verbose headers we don't care about, so we'll be using `bounded=` to remove them from the previews as well as to show just the subset of data we're working with.</span>|
|-----------------------------------------|

The [full data source can be downloaded here](https://github.com/mikeAdamss/tidychef/raw/main/tests/fixtures/xlsx/ons-oic.xlsx).

 For this example we'll be using the following tables:
 
 - The 4th table named "Table 1a".
 - The 5th table named "Table 1b".

 The principle difference between the tables is 1a is "seasonally adjusted" and 1b is not.

 For the sake of practicality we'll only be extracting observations down to row 12.

In [ ]:
from typing import List
from tidychef import acquire, preview
from tidychef.selection import XlsxSelectable

tables: List[XlsxSelectable] = acquire.xlsx.http("https://github.com/mikeAdamss/tidychef/raw/main/tests/fixtures/xlsx/ons-oic.xlsx", tables="Table 1a|Table 1b")
preview(tables[0], bounded="A1:H12")
preview(tables[1], bounded="A1:H12")

## An Iterated Extraction

In this example we're going to

- Iterate through the sheets
- Extract data from the two sheets in question - adding a column to indicate whether the data is seasonally adjusted.
- Join the data into a single TidyData putput.

In [ ]:
from tidychef import acquire, preview
from tidychef.direction import right, down
from tidychef.output import Column, TidyData
from tidychef.selection import XlsxSelectable

tables: List[XlsxSelectable] = acquire.xlsx.http("https://github.com/mikeAdamss/tidychef/raw/main/tests/fixtures/xlsx/ons-oic.xlsx", tables="Table 1a|Table 1b")

# An empty list to hold our tables
tidy_data_list = []

# Now iterate and extract
for table in tables:
    anchor = table.excel_ref("A5").label_as("Anchor Cell")
    observations = table.excel_ref("B7:H12").label_as("Observations")
    housing = anchor.fill(right).label_as("Housing")
    dataset_identifier_codes = housing.shift(down).label_as("Data Identifier Codes")
    period = anchor.shift(down(2)).expand(down).label_as("Period")

    # We're not gonna set a variable based on the contents of cell A1
    # this is what tells us if its SA of NSA
    a1_cell_value = table.excel_ref("A1").lone_value()
    is_seasonally_adjusted = "False" if "non-season" in a1_cell_value else "True"

    # Preview selections to sanity check
    # we'll include the anchor cell
    preview(anchor, observations, housing, dataset_identifier_codes, period, bounded="A1:H12")

    tidy_data = TidyData(
        observations,
        Column(housing.attach_directly(down)),
        Column(dataset_identifier_codes.attach_directly(down)),
        Column(period.attach_directly(right)),
        Column.constant("Seasonally Adjusted", is_seasonally_adjusted)
    )
    
    # Now append the tidy data for this sheet to our list
    tidy_data_list.append(tidy_data)
    

# concatenate the list and print our new output
all_tidy_data = TidyData.from_tidy_list(tidy_data_list)
print(all_tidy_data)
